In [ ]:
import pennylane as qml
import numpy as np

from QHyper.optimizers.CEM import CEM
from QHyper.optimizers.random import Random
from QHyper.optimizers.qml_gradient_descent import QmlGradientDescent
from QHyper.solvers.QAOA.core import QAOA

## TSP problem

In [ ]:
from QHyper.problems.TSP import TSPProblem

In [ ]:
tsp = TSPProblem(
    number_of_cities=3,
)

In [ ]:
solver = QAOA(
    problem=tsp,
    platform="pennylane",
    optimizer=QmlGradientDescent(200, qml.AdamOptimizer(stepsize=0.05)),
    layers=5,
    angles=[[0.5]*5, [0.5]*5],
    # mixer: str=,
    weights=[1, 1, 1],
    hyperoptimizer=CEM(
        epochs=5,
        samples_per_epoch=100,
        elite_frac=0.1
    ),
    # backend=
)

In [ ]:
value, params, weights = solver.solve()

In [ ]:
probs = solver.get_probs_func(weights)(params)

In [ ]:
solver.check_results(probs)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
solver.print_results(probs)

## Knapsack problem

In [ ]:
from QHyper.problems.Knapsack import Knapsack, KnapsackProblem

In [ ]:
knapsack = Knapsack(max_weight=2)
knapsack.set_knapsack([(2, 2), (3, 1)])
knapsack_qaoa = KnapsackProblem(knapsack)

In [ ]:
solver = QAOA(
    problem=knapsack_qaoa,
    platform="pennylane",
    optimizer=QmlGradientDescent(200, qml.AdamOptimizer(stepsize=0.05)),
    layers=5,
    angles=[[0.5]*5, [0.5]*5],
    # mixer: str=,
    weights=[2.72616422, 4.59436366, 4.46463072],
    # hyperoptimizer=CEM(),
    # backend=
)
value, params, weights = solver.solve()

In [ ]:
probs = solver.get_probs_func(weights)(params)

In [ ]:
solver.check_results(probs)

In [ ]:
solver.print_results(probs)